![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Experiment-Notebook.svg)
# Experiment Notebook - AutoAI Notebook v1.22.3


This notebook contains the steps and code to demonstrate support of AutoAI experiments in the Watson Machine Learning service. It introduces Python API commands for data retrieval, training experiments, persisting pipelines, testing pipelines, refining pipelines, and scoring the resulting model.

**Note:** Notebook code generated using AutoAI will execute successfully. If code is modified or reordered, there is no guarantee it will successfully execute. For details, see: <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Saving an Auto AI experiment as a notebook</a>


Some familiarity with Python is helpful. This notebook uses Python 3.10 and the `ibm-watsonx-ai` package.


## Notebook goals

The learning goals of this notebook are:
-  Defining an AutoAI experiment
-  Training AutoAI models 
-  Comparing trained models
-  Deploying the model as a web service
-  Scoring the model to generate predictions



## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Experiment configuration](#configuration)**<br>
&nbsp;&nbsp;[Experiment metadata](#metadata)<br>
**[Working with completed AutoAI experiment](#work)**<br>
&nbsp;&nbsp;[Get fitted AutoAI optimizer](#get)<br>
&nbsp;&nbsp;[Pipelines comparison](#comparison)<br>
&nbsp;&nbsp;[Get pipeline as a scikit-learn pipeline model](#get_pipeline)<br>
&nbsp;&nbsp;[Inspect pipeline](#inspect_pipeline)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Visualize pipeline model](#visualize)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Preview pipeline model as a Python code](#preview)<br>
**[Deploy and Score](#scoring)**<br>
&nbsp;&nbsp;[Working with spaces](#working_spaces)<br>
**[Running AutoAI experiment with Python API](#run)**<br>
**[Clean up](#cleanup)**<br>
**[Next steps](#next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watsonx-ai,
 - autoai-libs,
 - lale,
 - scikit-learn,
 - xgboost,
 - lightgbm,
 - snapml


In [1]:
#パッケージの導入はバージョンにもシビア

#!pip install ibm-watsonx-ai | tail -n 1
#!pip install autoai-libs==1.16.2 | tail -n 1
#!pip install -U 'lale>=0.7,<0.8' | tail -n 1
#!pip install scikit-learn==1.1.1 | tail -n 1
#!pip install xgboost==1.6.2 | tail -n 1
#!pip install lightgbm==3.3.2 | tail -n 1
#!pip install snapml==1.13.2 | tail -n 1

<a id="configuration"></a>
# Experiment configuration

<a id="metadata"></a>
## Experiment metadata
This cell defines the metadata for the experiment, including: training_data_references, training_result_reference, experiment_metadata.

In [2]:
from ibm_watsonx_ai.helpers import DataConnection
from ibm_watsonx_ai.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='e074bfe1-a54f-482f-a461-e85fd7dcf70d'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/19f9b29f-8614-40bc-aa6a-6d8bdf8c6e5d/wml_data/3be8693a-e313-463d-a148-064633a00ed8/data/automl',
        model_location='auto_ml/19f9b29f-8614-40bc-aa6a-6d8bdf8c6e5d/wml_data/3be8693a-e313-463d-a148-064633a00ed8/data/automl/model.zip',
        training_status='auto_ml/19f9b29f-8614-40bc-aa6a-6d8bdf8c6e5d/wml_data/3be8693a-e313-463d-a148-064633a00ed8/training-status.json'
    )
)

In [3]:
#GUIで実行したエクスペリメントの情報
experiment_metadata = dict(
    prediction_type='binary',
    prediction_column='今回販促結果',
    holdout_size=0.1,
    scoring='roc_auc',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    deployment_url='https://jp-tok.ml.cloud.ibm.com',
    project_id='b946aa42-c3bf-4f99-962f-5562a7415a85',
    positive_label='成功',
    drop_duplicates=True,
    include_batched_ensemble_estimators=[],
    feature_selector_mode='auto'
)

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [4]:
#ここにＡＰＩキーを書く
api_key = 'PUT_YOUR_APIKEY_HERE'

In [5]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

<a id="work"></a>


# Working with the completed AutoAI experiment

This cell imports the pipelines generated for the experiment. The best pipeline will be saved as a model.

<a id="get"></a>


## Get fitted AutoAI optimizer

In [6]:
#GUIで実行したエクスペリメントを取得
from ibm_watsonx_ai.experiment import AutoAI

pipeline_optimizer = AutoAI(wml_credentials, project_id=experiment_metadata['project_id']).runs.get_optimizer(metadata=experiment_metadata)

Use `get_params()` to retrieve configuration parameters.

In [7]:
pipeline_optimizer.get_params()

{'name': '営業成約予測',
 'desc': '',
 'prediction_type': 'classification',
 'prediction_column': '今回販促結果',
 'prediction_columns': None,
 'timestamp_column_name': None,
 'scoring': 'roc_auc',
 'holdout_size': 0.1,
 'max_num_daub_ensembles': 2,
 't_shirt_size': 'a6c4923b-b8e4-444c-9f43-8a7ec3020110',
 'train_sample_rows_test_size': None,
 'include_only_estimators': None,
 'include_batched_ensemble_estimators': None,
 'backtest_num': None,
 'lookback_window': None,
 'forecast_window': None,
 'backtest_gap_length': None,
 'cognito_transform_names': None,
 'csv_separator': ',',
 'excel_sheet': None,
 'encoding': 'utf-8',
 'positive_label': None,
 'drop_duplicates': True,
 'outliers_columns': None,
 'text_processing': None,
 'word2vec_feature_number': None,
 'daub_give_priority_to_runtime': None,
 'text_columns_names': None,
 'sampling_type': None,
 'sample_size_limit': None,
 'sample_rows_limit': None,
 'sample_percentage_limit': None,
 'number_of_batch_rows': None,
 'n_parallel_data_connections

<a id="comparison"></a>
## Pipelines comparison

Use the `summary()` method to list trained pipelines and evaluation metrics information in
the form of a Pandas DataFrame. You can use the DataFrame to compare all discovered pipelines and select the one you like for further testing.

In [8]:
summary = pipeline_optimizer.summary()
best_pipeline_name = list(summary.index)[0]
summary

,Enhancements,Estimator,training_roc_auc_(optimized),holdout_average_precision,holdout_log_loss,training_accuracy,holdout_roc_auc,training_balanced_accuracy,training_f1,holdout_precision,training_average_precision,training_log_loss,holdout_recall,training_precision,holdout_accuracy,holdout_balanced_accuracy,training_recall,holdout_f1
Pipeline Name,,,,,,,,,,,,,,,,,,
Pipeline_5,,SnapBoostingMachineClassifier,0.900147,0.639144,0.311249,0.850541,0.927689,0.813275,0.546136,0.396552,0.533841,0.320802,0.867925,0.426124,0.830022,0.846462,0.764642,0.544379
Pipeline_2,HPO,XGBClassifier,0.914173,0.602386,0.218805,0.897493,0.917406,0.667669,0.456521,0.628571,0.555639,0.225118,0.415094,0.603419,0.902870,0.691297,0.367633,0.500000
Pipeline_1,,XGBClassifier,0.908120,0.598384,0.220766,0.898968,0.915472,0.690361,0.491652,0.628571,0.570720,0.233275,0.415094,0.597610,0.902870,0.691297,0.418027,0.500000
Pipeline_6,HPO,SnapBoostingMachineClassifier,0.905047,0.575023,0.346416,0.841445,0.914387,0.823603,0.543189,0.398305,0.553501,0.337899,0.886792,0.413059,0.830022,0.854646,0.800334,0.549708
Pipeline_4,"HPO, FE, HPO",XGBClassifier,0.910521,0.550208,0.222258,0.902409,0.913231,0.678634,0.478958,0.586207,0.581469,0.223956,0.320755,0.638074,0.894040,0.645377,0.386501,0.414634
Pipeline_3,"HPO, FE",XGBClassifier,0.907417,0.544790,0.226012,0.899705,0.912594,0.675299,0.471756,0.527778,0.553683,0.228169,0.358491,0.617726,0.887417,0.657995,0.382334,0.426966
Pipeline_7,"HPO, FE",SnapBoostingMachineClassifier,0.904850,0.551891,0.367636,0.840216,0.904080,0.824739,0.542313,0.387931,0.533336,0.338973,0.849057,0.410457,0.825607,0.835778,0.804554,0.532544
Pipeline_8,"HPO, FE, HPO",SnapBoostingMachineClassifier,0.909709,0.518166,0.374965,0.839971,0.903750,0.834606,0.548127,0.391304,0.566787,0.342132,0.849057,0.410914,0.827815,0.837028,0.827628,0.535714


<a id="get_pipeline"></a>
### Get pipeline as a scikit-learn pipeline model

After you compare the pipelines, download and save a scikit-learn pipeline model object from the
AutoAI training job.

**Tip:** To get a specific pipeline, pass the pipeline name in:
```
pipeline_optimizer.get_pipeline(pipeline_name=pipeline_name)
```

In [9]:
#pipeline_model = pipeline_optimizer.get_pipeline()
#最高ランクのパイプラインを取得
pipeline_model = pipeline_optimizer.get_pipeline(pipeline_name='Pipeline_2')

#pickleでローカルに保存
import pickle

with open('pipeline_model.pickle', mode='wb') as fo:
  pickle.dump(pipeline_model, fo)

Next, check the importance of features for selected pipeline.

In [10]:
pipeline_optimizer.get_pipeline_details()['features_importance']

,features_importance
最終通話秒数,0.44
最終通話月,0.11
前回販促結果,0.10
連絡手段,0.07
年齢,0.06
住宅ローン,0.05
平均残高,0.05
前回販促後_経過日数,0.04
最終通話日,0.03
通話回数_販促中,0.02


**Tip:** If you want to check all the details of the model evaluation metrics, use:
```
pipeline_optimizer.get_pipeline_details()
```

<a id="inspect_pipeline"></a>
## Inspect pipeline

<a id="visualize"></a>
### Visualize pipeline model

Preview pipeline model stages as a graph. Each node's name links to a detailed description of the stage.


In [11]:
#実行のためにはgraphvizが必要
#pipeline_model.visualize()

<a id="preview"></a>
### Preview pipeline model as a Python code
In the next cell, you can preview the saved pipeline model as a Python code.  
You can review the exact steps used to create the model.

**Note:** If you want to get sklearn representation, add the following parameter to the `pretty_print` call: `astype='sklearn'`.

In [12]:
#モデルのPythonコードを表示。スコアリングにはimport文が必要
pipeline_model.pretty_print(combinators=False, ipython_display=True)

```python
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from lale.operators import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from lale.operators import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from xgboost import XGBClassifier

numpy_column_selector_0 = NumpyColumnSelector(
    columns=[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 14, 15]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "int_num", "char_str", "char_str", "char_str", "char_str", "char_str",
        "char_str", "char_str", "int_num", "char_str", "int_num", "int_num",
        "char_str",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], [], [], [], [], [], [], [], [], [], [], []],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
    encoding="ordinal",
    categories="auto",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[5, 11, 13])
float_str2_float = FloatStr2Float(
    dtypes_list=["int_num", "int_num", "int_num"],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 14, 15, 5, 11, 13,
    ],
)
xgb_classifier = XGBClassifier(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    gpu_id=-1,
    grow_policy="depthwise",
    interaction_constraints="",
    learning_rate=0.0258858021094187,
    max_bin=256,
    max_cat_to_onehot=4,
    max_delta_step=0,
    max_depth=6,
    max_leaves=0,
    min_child_weight=16,
    missing=float("nan"),
    monotone_constraints="()",
    n_estimators=379,
    n_jobs=4,
    num_parallel_tree=1,
    predictor="auto",
    random_state=33,
    reg_alpha=1,
    reg_lambda=0.5044001741973043,
    sampling_method="uniform",
    scale_pos_weight=1,
    subsample=0.7522703370047685,
    tree_method="hist",
    validate_parameters=1,
    verbosity=0,
    nthread=4,
    silent=True,
    seed=33,
)
pipeline = make_pipeline(union, numpy_permute_array, xgb_classifier)
```

### Calling the `predict` method
If you want to get a prediction by using the pipeline model object, call `pipeline_model.predict()`.

**Note:** If you want to work with a pure sklearn model:
 - add the following parameter to the `get_pipeline` call: `astype='sklearn'`,
 - or `scikit_learn_pipeline = pipeline_model.export_to_sklearn_pipeline()`

<a id="scoring"></a>
## Deploy and Score

In this section you will learn how to deploy and score the model as a web service.

<a id="working_spaces"></a>
### Working with spaces

In this section you will specify a deployment space to organize assets and then deploy and score the model. If you do not have an existing space, you can use the [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create a new space. Follow these steps:

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press **Create**
- Copy `space_id` and paste it below

**Tip**: You can also use the API to prepare the space for your work. Learn more [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign or update space ID below.

### Deployment creation

target_space_id = "PUT_YOUR_TARGET_SPACE_ID_HERE"

from ibm_watsonx_ai.deployment import WebService

service = WebService(
    source_wml_credentials=wml_credentials,
    target_wml_credentials=wml_credentials,
    source_project_id=experiment_metadata['project_id'],
    target_space_id=target_space_id
)
service.create(
    model=best_pipeline_name,
    metadata=experiment_metadata,
    deployment_name='Best_pipeline_webservice'
)

Use the `print` method for the deployment object to show basic information about the service: 

print(service)

To show all available information about the deployment, use the `.get_params()` method.

service.get_params()

### Scoring of webservice
You can make a scoring request by calling `score()` on the deployed pipeline.

If you want to work with the web service in an external Python application, follow these steps to retrieve the service object:

 - Initialize the service by `service = WebService(target_wml_credentials=wml_credentials,target_space_id=experiment_metadata['space_id'])`
 - Get deployment_id: `service.list()`
 - Get webservice object: `service.get('deployment_id')`

After that you can call `service.score(score_records_df)` method. The `score()` method accepts `pandas.DataFrame` objects. 

<a id="cleanup"></a>
### Deleting deployment
You can delete the existing deployment by calling the `service.delete()` command.
To list the existing web services, use the `service.list()` method.

<a id="run"></a>

## Running the AutoAI experiment with Python API

If you want to run the AutoAI experiment using the Python API, follow these steps. The experiment settings were generated basing on parameters set in the AutoAI UI.


```
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(wml_credentials, project_id=experiment_metadata['project_id'])
```

```
OPTIMIZER_NAME = 'custom_name'
```

```
from ibm_watsonx_ai.helpers import DataConnection
from ibm_watsonx_ai.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='e074bfe1-a54f-482f-a461-e85fd7dcf70d'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/19f9b29f-8614-40bc-aa6a-6d8bdf8c6e5d/wml_data/3be8693a-e313-463d-a148-064633a00ed8/data/automl',
        model_location='auto_ml/19f9b29f-8614-40bc-aa6a-6d8bdf8c6e5d/wml_data/3be8693a-e313-463d-a148-064633a00ed8/data/automl/model.zip',
        training_status='auto_ml/19f9b29f-8614-40bc-aa6a-6d8bdf8c6e5d/wml_data/3be8693a-e313-463d-a148-064633a00ed8/training-status.json'
    )
)
```

The new pipeline optimizer will be created and training will be triggered.

```
pipeline_optimizer = experiment.optimizer(
    name=OPTIMIZER_NAME,
    prediction_type=experiment_metadata['prediction_type'],
    prediction_column=experiment_metadata['prediction_column'],
    scoring=experiment_metadata['scoring'],
    holdout_size=experiment_metadata['holdout_size'],
    csv_separator=experiment_metadata['csv_separator'],
    positive_label=experiment_metadata['positive_label'],
    drop_duplicates=experiment_metadata['drop_duplicates'],
    include_batched_ensemble_estimators=experiment_metadata['include_batched_ensemble_estimators'],
    feature_selector_mode=experiment_metadata['feature_selector_mode'],
)
```

```
pipeline_optimizer.fit(
    training_data_references=training_data_references,
    training_results_reference=training_result_reference,
)
```


<a id="next_steps"></a>
# Next steps
You successfully completed this notebook!
You learned how to use ibm-watsonx-ai to run and explore AutoAI experiments.
Check out the official [AutoAI site](https://www.ibm.com/cloud/watson-studio/autoai) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>  

___